In [18]:
# Setup and imports
import os, json
from pathlib import Path
from typing import List, Tuple, Union

from dotenv import load_dotenv
from pypdf import PdfReader
from openai import OpenAI

load_dotenv()
client = OpenAI()

ROOT = Path.cwd()
ME_DIR = ROOT / "me"
ABOUT_PDF = ME_DIR / "about_business.pdf"
SUMMARY_TXT = ME_DIR / "summary.txt"
SYSTEM_PROMPT_FILE = ROOT / "system_prompt.txt"

DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")


def _read_pdf_text(pdf_path: Path, max_chars: int = 25000) -> str:
    if not pdf_path.exists():
        return "[about_business.pdf not found]"
    chunks = []
    try:
        reader = PdfReader(str(pdf_path))
        for p in reader.pages:
            chunks.append(p.extract_text() or "")
    except Exception as e:
        chunks.append(f"[PDF read error: {e}]")
    return ("\n".join(chunks))[:max_chars]


def _read_text(path: Path, placeholder: str, max_chars: int = 24000) -> str:
    if not path.exists():
        return placeholder
    try:
        return path.read_text(encoding="utf-8")[:max_chars]
    except Exception as e:
        return f"[read error {path.name}: {e}]"


In [19]:
# Load system prompt and business docs
SYSTEM_PROMPT = _read_text(
    SYSTEM_PROMPT_FILE,
    placeholder=(
        "You are a helpful business assistant. Use the provided business files "
        "to answer questions, stay on-brand, and keep answers concise."
    ),
)

BUSINESS_DOCS = {
    "about_business.pdf": _read_pdf_text(ABOUT_PDF),
    "summary.txt": _read_text(SUMMARY_TXT, "[summary.txt not found]"),
}


def build_messages(user_msg: str, history: List[Tuple[str, str]]) -> list:
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    files_blob = (
        "Business files (reference only; do not reveal raw text unless asked):\n\n"
        "[about_business.pdf]\n" + BUSINESS_DOCS["about_business.pdf"][:12000] + "\n\n"
        "[summary.txt]\n" + BUSINESS_DOCS["summary.txt"][:8000]
    )
    messages.append({"role": "system", "content": files_blob})

    for u, a in history or []:
        if u:
            messages.append({"role": "user", "content": u})
        if a:
            messages.append({"role": "assistant", "content": a})

    messages.append({"role": "user", "content": user_msg})
    return messages


In [20]:
# Tools schema and local dispatch
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Save a potential customer's contact info and notes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string"},
                    "name": {"type": "string"},
                    "message": {"type": "string"},
                },
                "required": ["email", "name", "message"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Log an unanswered question for follow-up by the team.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"},
                },
                "required": ["question"],
            },
        },
    },
]

from pathlib import Path as _Path
import csv as _csv, json as _json, datetime as _datetime
DATA_DIR = _Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)
LEADS_CSV = DATA_DIR / "leads.csv"
FEEDBACK_JSONL = DATA_DIR / "feedback.jsonl"

def _record_customer_interest(email: str, name: str, message: str) -> str:
    LEADS_CSV.parent.mkdir(parents=True, exist_ok=True)
    is_new = not LEADS_CSV.exists()
    with LEADS_CSV.open("a", newline="") as f:
        writer = _csv.writer(f)
        if is_new:
            writer.writerow(["timestamp", "name", "email", "message"])  # header
        writer.writerow([_datetime.datetime.utcnow().isoformat(), name, email, message])
    print(f"[LEAD] {name} <{email}> - {message}")
    return "Thanks! We've recorded your interest and will reply soon."

def _record_feedback(question: str) -> str:
    FEEDBACK_JSONL.parent.mkdir(parents=True, exist_ok=True)
    entry = {
        "timestamp": _datetime.datetime.utcnow().isoformat(),
        "question": question,
    }
    with FEEDBACK_JSONL.open("a") as f:
        f.write(_json.dumps(entry) + "\n")
    print(f"[FEEDBACK] {question}")
    return "Got it - I've logged your question for the team."

def _call_local_tool(tool_name: str, args_json: str) -> str:
    try:
        args = json.loads(args_json or "{}")
    except json.JSONDecodeError as e:
        return f"[Tool args parse error] {e}"
    try:
        if tool_name == "record_customer_interest":
            return _record_customer_interest(**args)
        if tool_name == "record_feedback":
            return _record_feedback(**args)
        return f"[Unknown tool] {tool_name}"
    except Exception as e:
        return f"[Tool error] {e}"


In [21]:
# reply() with tool-calling and fallback to log unknown questions

def reply(user_message: str, history: List[Tuple[str, str]] | None = None) -> str:
    if history is None:
        history = []

    messages = build_messages(user_message, history)

    # First pass: allow tools
    resp = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
        temperature=0.4,
    )
    msg = resp.choices[0].message

    if getattr(msg, "tool_calls", None):
        tool_msgs = []
        for tc in msg.tool_calls:
            result = _call_local_tool(tc.function.name, tc.function.arguments)
            tool_msgs.append(
                {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "name": tc.function.name,
                    "content": result,
                }
            )
        follow = client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages + [{"role": "assistant", "tool_calls": msg.tool_calls}] + tool_msgs,
            temperature=0.4,
        )
        return follow.choices[0].message.content

    # Fallback: gentle policy to use tools when appropriate (not overly strict)
    followup_instructions = (
        "Policy for tools: \n"
        "- If the question CAN be answered confidently from the provided business files, answer directly (no tools).\n"
        "- If the question CANNOT be answered confidently from those files (info missing/ambiguous), then call record_feedback(question) before replying.\n"
        "- If the user expresses interest in lessons/consults or asks to be contacted (e.g., 'interested', 'sign up', 'book', 'contact me'), use record_customer_interest(email, name, message).\n"
        "- If email/name are not provided, ask for them first; only call the tool after you have them."
    )
    follow2 = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=messages + [
            {"role": "assistant", "content": msg.content or ""},
            {"role": "system", "content": followup_instructions},
        ],
        tools=TOOLS,
        tool_choice="auto",
        temperature=0.4,
    )
    msg2 = follow2.choices[0].message

    if getattr(msg2, "tool_calls", None):
        tool_msgs2 = []
        for tc in msg2.tool_calls:
            result = _call_local_tool(tc.function.name, tc.function.arguments)
            tool_msgs2.append(
                {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "name": tc.function.name,
                    "content": result,
                }
            )
        follow3 = client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages
            + [{"role": "assistant", "content": msg.content or ""}]
            + [{"role": "assistant", "tool_calls": msg2.tool_calls}]
            + tool_msgs2,
            temperature=0.4,
        )
        return follow3.choices[0].message.content

    return msg.content


In [22]:
# Quick test
print(reply("What are your enterprise discounts?"))


C:\Users\Mawlawui\AppData\Local\Temp\ipykernel_17252\1643643839.py:56: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": _datetime.datetime.utcnow().isoformat(),


[FEEDBACK] What are your enterprise discounts?
If you have any other questions or if you're interested in our tutoring services, please share your name and email, along with a short note! I'm here to help.


In [23]:
print(reply("what is you services"))

At OrbitCoach Tutors, we offer a range of services to help students master challenging subjects, including:

1. **1-to-1 Tutoring**: Personalized sessions in Math, Physics, and Computer Science, available live or online.
2. **10-Minute Concept Boost**: Quick micro-sessions designed to clarify specific concepts.
3. **Homework Clinics & Exam Prep Packs**: Support for homework and preparation for upcoming exams.
4. **Progress Dashboards**: Tools for both parents and students to track learning progress.

If you're interested in any of these services, please share your name and email, and I can provide more information!


In [24]:
print(reply("Hi! Do you do Physics micro‑sessions?"))

Yes, we offer fast "10-minute concept boost" micro-sessions for Physics! These sessions are designed to help students quickly grasp specific concepts. If you're interested in scheduling a session, please share your name and email, and I can assist you further!


In [25]:
print(reply("I'm interested. My name is Lina, lina@example.com — can we do Sunday?"))

C:\Users\Mawlawui\AppData\Local\Temp\ipykernel_17252\1643643839.py:49: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  writer.writerow([_datetime.datetime.utcnow().isoformat(), name, email, message])


[LEAD] Lina <lina@example.com> - Interested in scheduling a session on Sunday.
Thank you, Lina! I've noted your interest in scheduling a session on Sunday. Our team will reach out to you shortly to confirm the details. If you have any specific subjects or topics you'd like to focus on, feel free to let us know!
